In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import os

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])

mycursor = mydb.cursor()

In [4]:

query = "select userid from usr_applied_jobs_info where unixtimestamp>1707374166"

mycursor.execute(query)
result = mycursor.fetchall()

In [5]:
active_users = pd.DataFrame(result,columns=['userid'])

In [6]:
active_users.drop_duplicates(inplace=True)

In [7]:
active_users

,userid
0,1780851
1,1614881
2,939714
3,1684573
4,2142750
...,...
1371438,2822603
1371515,2821276
1371517,2700390
1371613,2821472


In [8]:
ulist = tuple(active_users['userid'].tolist())

In [9]:

query = f"select id, resume_last_updated, resume_first_uploaded from profile_info where id in {ulist}"

mycursor.execute(query)
result = mycursor.fetchall()

In [10]:
rdata = pd.DataFrame(result,columns=['id','last_upload','first_upload'])

In [18]:
rdata.shape

(101234, 4)

In [11]:
# rdata['first_upload'].drop('0000-00-00 00:00:00') 

In [12]:
rdata['first_upload'] = pd.to_datetime(rdata['first_upload'],errors='coerce')
rdata['last_upload'] = pd.to_datetime(rdata['last_upload'])

# Calculate the difference in years
rdata['year_difference'] = rdata['last_upload'] - rdata['first_upload']
rdata['year_difference'] = rdata['year_difference'] / pd.Timedelta(days=365.25)

# To round the years to a sensible number of decimal places, e.g., 2 decimal places
rdata['year_difference'] = rdata['year_difference'].round(2)


In [19]:
rdata['now'] = pd.Timestamp.now()  

In [21]:
rdata['year_difference'] = rdata['now'] - rdata['last_upload']
rdata['year_difference'] = rdata['year_difference'] / pd.Timedelta(days=365.25)
rdata['year_difference'] = rdata['year_difference'].round(2)

In [22]:
select = rdata[rdata['year_difference']>3]

In [23]:
select

,id,last_upload,first_upload,year_difference,now
6,123,2019-12-07 11:30:53,2012-09-22 16:54:17,4.26,2024-03-11 11:30:20.184937
7,135,2018-09-26 11:34:04,2013-02-24 19:24:44,5.46,2024-03-11 11:30:20.184937
34,886,2018-09-24 02:29:09,2013-11-25 14:52:35,5.46,2024-03-11 11:30:20.184937
67,2014,2015-08-18 13:46:20,2014-02-21 14:21:50,8.56,2024-03-11 11:30:20.184937
68,2021,2014-07-10 13:44:05,2014-07-10 13:44:05,9.67,2024-03-11 11:30:20.184937
...,...,...,...,...,...
53205,1985580,2021-03-06 11:26:34,2021-03-06 11:26:35,3.01,2024-03-11 11:30:20.184937
53209,1985752,2021-03-06 15:16:11,2021-03-06 15:16:11,3.01,2024-03-11 11:30:20.184937
53246,1987530,2021-03-08 22:19:34,2021-03-08 22:19:34,3.01,2024-03-11 11:30:20.184937
53254,1988121,2021-03-09 16:42:11,2021-03-09 16:42:11,3.01,2024-03-11 11:30:20.184937


In [24]:
from user_info import get_user_info

In [25]:
r_text = {}
for uid in tqdm(select['id'].tolist()):
    response = get_user_info(uid)
    try:
        r_text[uid] = response['resume'][0]
    except:
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2457/2457 [00:17<00:00, 137.58it/s]


In [26]:
import json
with open('../custom_data/relevant_users_feb24.json','w') as f:
    json.dump(r_text,f)